# Mastermind AI Game Assistant - Multi-Modal LLM Project

## Project Overview

This notebook implements an AI-powered assistant for the classic code-breaking game **Mastermind**. I have worked across frontend frameworks (react, angular), backend API centric applications (Java SpringBoot, Node) and everything in between, and am interested in how quickly this can be achieved with the gradio UX, incorporating LLM for game logic next moves... etc.. and can include concepts covered including **tool use**, **function calling**, **multi-modal interactions**, and **stateful conversation management**.

---

## About Mastermind

Mastermind is a code-breaking logic game where:
- The computer creates a secret code of coloured pegs (typically 4 colors chosen from 6-8 options)
- The player makes guesses to crack the code
  - duplicate colours are allowed
  - llm provides next move analysis with text to speech for summary
- After each guess, feedback is provided:
  - **Black pegs**: Correct colour in the correct position
  - **White pegs**: Correct colour in the wrong position
- The goal is to crack the code in the fewest guesses possible (usually 10-12 attempts)





In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

# Import the game logic and UI
from mastermind_game import (
    load_css,
    MastermindGame, 
    create_game_board_html
)

from mastermind_llm import (
    analyze_guess_with_llm,
    generate_game_history_prompt,
    generate_user_guess_prompt,
    generate_game_state_prompt,
    call_openai_api,
    call_openai_api_with_messages,
    talker
)



### Load environment variables

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyA1


### Setup models via OpenAI and OLLAMA for local models

In [3]:
MY_MODEL = 'gpt-4o-mini'
#MY_MODEL= 'gemma3:4b' # OLLAMA

class Config:
    SDK = OpenAI()
    API_KEY = os.getenv("OPENAI_API_KEY")
    MODEL = MY_MODEL
    VOICE_MODEL = "tts-1"
    VOICE = "onyx"
    TEMPERATURE = 0.7
    BASE_URL =  None #  "http://localhost:11434/v1" for ollama... but no voice
    
config = Config()


### Initialise LLM and Game constructs
* Initialise game
* Allow prompts for expirementation (externalise from code)

In [4]:
# game rules, ensure the rule context is not lost across multiple contexts
game_rules_prompt = (
"""
** MASTERMIND GAME RULES are as follows: **
- The computer creates a secret code of four pegs, each chosen from six possible colors: Red, Blue, Green, Yellow, Orange, Purple.
- Duplicates are allowed (the same color may appear multiple times in the code).
- The player makes guesses to crack the code.
- After each guess:
    - a COUNT of Black pegs are given for each peg that's the correct color and in the correct position.
    - a COUNT of White pegs are given for each correct color that is in the wrong position.
    - Number of black and white pegs does NOT reveal the positions.
- The goal is to guess the code in as few attempts as possible.
"""
)

# prompt for move analysis
move_analysis_system_prompt = """
You are an expert Mastermind game strategist and analyst.
{game_rules_prompt}\n
Your role is to:
1. Analyze game states using logical deduction and probability theory
2. Provide clear, actionable strategic advice
3. Explain your reasoning in a structured, easy-to-understand way
4. Balance between information gathering and solution finding
5. Consider both offensive (finding the code) and defensive (minimizing guesses) strategies

Be concise but thorough. Always end with a clear recommendation.
"""

move_analysis_user_prompt = """
Analyze this Mastermind game situation and proposed guess.
{game_rules_prompt}
{generate_game_history_prompt}
{generate_user_guess_prompt}
{generate_game_state_prompt}
"""

# system prompt for user feedback loop on analysis
system_question_prompt_template = (
    "You are an expert Mastermind game strategist and analyst. Your role is to:"
    "1. Understand the rules. {game_rules_prompt}\n"
    "2. When a user a question to challenge your move analysis about a previous Mastermind guess"
    "Explain or justify the feedback according to Mastermind rules in the context of the users challenge."
    "Be concise but thorough. Always end with a clear recommendation."
)

# user prompt for challenging move analysis results with context
user_question_prompt_template = (
    "Here is the last feedback given about a Mastermind guess:\n"
    "{previous_feedback}\n"
    "The user asks for clarification: {user_question}\n"
    "Explain or justify the feedback according to Mastermind rules."
)


In [5]:
game = MastermindGame()
game.start_new_game()
current_guess_state = [None, None, None, None]
conversation_history = []

def question_feedback(previous_feedback, user_question, current_question_history):

    system_message_content = system_question_prompt_template.format(
        game_rules_prompt=game_rules_prompt
    )
    messages_for_llm = [{"role": "system", "content": system_message_content}]

    # Add previous analysis as 'assistant' message (if available)
    if previous_feedback:
        messages_for_llm.append({"role": "assistant", "content": f"Previous analysis: {previous_feedback}"})

    # Add the conversation history
    messages_for_llm.extend(current_question_history or []) # Ensure it's a list

    # Add the current user's challenge/question
    formatted_user_question_content = user_question_prompt_template.format(
        previous_feedback=previous_feedback,
        user_question=user_question
    )
    messages_for_llm.append({"role": "user", "content": formatted_user_question_content})

    print(f"Messages sent to LLM: {messages_for_llm}") # Debugging

    answer = call_openai_api_with_messages(
        messages_list=messages_for_llm,   
        config=config
    )

    # 6. Update the conversation history (for Gradio state)
    updated_question_history = (current_question_history or []) + \
        [{"role": "user", "content": formatted_user_question_content},
         {"role": "assistant", "content": answer}]

    return answer, updated_question_history

def analyze_current_guess():
    global current_guess_state
    
    user_prompt = move_analysis_user_prompt.format(
        game_rules_prompt=game_rules_prompt,
        generate_game_history_prompt=generate_game_history_prompt(game),
        generate_user_guess_prompt=generate_user_guess_prompt(current_guess_state),
        generate_game_state_prompt=generate_game_state_prompt(game)
    )
    
    system_prompt = move_analysis_system_prompt.format( 
        game_rules_prompt=game_rules_prompt)
    
    detailed, summary = analyze_guess_with_llm(
        game, 
        current_guess_state, 
        system_prompt,
        user_prompt,
        config
    )
    
    audio = talker(summary, config)
    
    return detailed, audio

def update_position(pos_idx, new_color):
    """Update a position with selected color"""
    global current_guess_state
    if new_color:
        current_guess_state[pos_idx] = new_color
    return create_game_board_html(game, current_guess_state)

def submit_guess():
    """Handle guess submission"""
    global current_guess_state
    
    if game.game_over:
        return create_game_board_html(game, current_guess_state, "Game is over! Click 'New Game' to play again.")
    
    guess = current_guess_state.copy()
    message = game.make_guess(guess)
    
    if all(c is not None for c in current_guess_state):
        current_guess_state = [None, None, None, None]
    
    return create_game_board_html(game, current_guess_state, message)

def new_game():
    """Start a new game"""
    global current_guess_state
    current_guess_state = [None, None, None, None]
    message = game.start_new_game()
    board_html = create_game_board_html(game, current_guess_state)
    return board_html, message


### Gradio layout
Layout the components in a vertical style gradio row layout
* game board
* LLM move analysers and summary to speech
* game colour selection

In [6]:
with gr.Blocks(title="Mastermind AI Assistant", theme=gr.themes.Soft()) as demo:
    # Load external CSS
    css = load_css("mastermind_styles.css")
    gr.HTML(css)
    
    with gr.Group():
        with gr.Row():
            gr.Markdown(f"""
            # 🎯 Mastermind AI Game Assistant
            **LLM Provider:** {config.MODEL.upper()} {f"({config.MODEL})" if config.MODEL else "ooops"}
            
            **How to play:** Select a color for each position, then click Submit Guess!
            - ⚫ **Black** = Right color, right position  
            - ⚪ **White** = Right color, wrong position
            """)
    
    with gr.Group():          
        with gr.Row():
            game_board = gr.HTML(value=create_game_board_html(game, current_guess_state))
            
    with gr.Group():
            
        with gr.Row():
            new_game_btn = gr.Button("🔄 New Game", size="sm")
            hint_btn = gr.Button("💡 Quick Hint", size="sm")
            

        with gr.Row():
            gr.Markdown("### 🎨 Make Your Guess")

        with gr.Row():
            with gr.Row():
                color1 = gr.Dropdown([""] + MastermindGame.COLORS, label="Position 1", value="")
                color2 = gr.Dropdown([""] + MastermindGame.COLORS, label="Position 2", value="")
                color3 = gr.Dropdown([""] + MastermindGame.COLORS, label="Position 3", value="")
                color4 = gr.Dropdown([""] + MastermindGame.COLORS, label="Position 4", value="")

        with gr.Row():  
            with gr.Row():
                submit_btn = gr.Button("🎯 Submit Guess", variant="primary")
                analyze_btn = gr.Button("🤖 Analyze Guess", variant="secondary")
                
    with gr.Group():
                
        with gr.Row():

            detailed_output = gr.Textbox(
                label="Detailed Analysis",
                interactive=False,
                lines=2,
                placeholder="Click 'Analyze' for AI feedback...",
            )

            audio_output = gr.Audio(autoplay=True)
            
        with gr.Row():
            user_question = gr.Textbox(label="Ask about this feedback", lines=2)
            answer_output = gr.Textbox(label="LLM's Answer", lines=2, interactive=False)
            challenge_btn = gr.Button("🔄 Challenge Analysis", size="sm")

        color1.change(fn=lambda c: update_position(0, c), inputs=[color1], outputs=[game_board])
        color2.change(fn=lambda c: update_position(1, c), inputs=[color2], outputs=[game_board])
        color3.change(fn=lambda c: update_position(2, c), inputs=[color3], outputs=[game_board])
        color4.change(fn=lambda c: update_position(3, c), inputs=[color4], outputs=[game_board])

        analyze_btn.click(fn=analyze_current_guess, outputs=[detailed_output, audio_output])

        def wrapped_submit():
            board_html = submit_guess()
            return create_game_board_html(game, current_guess_state), "", "", "", ""

        submit_btn.click(fn=wrapped_submit, outputs=[game_board, color1, color2, color3, color4])
        
        challenge_btn.click(fn=question_feedback, inputs=[detailed_output, user_question], outputs=[answer_output])
        
        new_game_btn.click(fn=new_game, outputs=[game_board]).then(
            fn=lambda: ("", "", "", ""), outputs=[color1, color2, color3, color4]
        )

/Users/georgewiles/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/utils.py:1052: UserWarning: Expected 3 arguments for function <function question_feedback at 0x123eaaca0>, received 2.
  warnings.warn(
/Users/georgewiles/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/utils.py:1056: UserWarning: Expected at least 3 arguments for function <function question_feedback at 0x123eaaca0>, received 2.
  warnings.warn(


In [7]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


/Users/georgewiles/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/helpers.py:1060: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")


Messages sent to LLM: [{'role': 'system', 'content': "You are an expert Mastermind game strategist and analyst. Your role is to:1. Understand the rules. \n** MASTERMIND GAME RULES are as follows: **\n- The computer creates a secret code of four pegs, each chosen from six possible colors: Red, Blue, Green, Yellow, Orange, Purple.\n- Duplicates are allowed (the same color may appear multiple times in the code).\n- The player makes guesses to crack the code.\n- After each guess:\n    - a COUNT of Black pegs are given for each peg that's the correct color and in the correct position.\n    - a COUNT of White pegs are given for each correct color that is in the wrong position.\n    - Number of black and white pegs does NOT reveal the positions.\n- The goal is to guess the code in as few attempts as possible.\n\n2. When a user a question to challenge your move analysis about a previous Mastermind guessExplain or justify the feedback according to Mastermind rules in the context of the users ch